In [78]:
import pandas as pd
import numpy as np
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [79]:
df = pd.read_csv('Avg.csv')
df.head()

,time,latitude,longitude,temperature
0,1750-01-01,26.5,-122.5,NaN
1,1750-01-01,26.5,-121.5,NaN
2,1750-01-01,26.5,-120.5,NaN
3,1750-01-01,26.5,-119.5,NaN
4,1750-01-01,26.5,-118.5,NaN


In [80]:
df.shape

(3743168, 4)

In [81]:
df['time'] = pd.to_datetime(df['time'])

In [82]:
df['time'].min(), df['time'].max()

(Timestamp('1750-01-01 00:00:00'), Timestamp('2022-08-01 00:00:00'))

In [83]:
df['time'].describe(datetime_is_numeric=True)

count                          3743168
mean     1886-04-16 10:37:15.696819712
min                1750-01-01 00:00:00
25%                1818-02-22 00:00:00
50%                1886-04-16 00:00:00
75%                1954-06-08 12:00:00
max                2022-08-01 00:00:00
Name: time, dtype: object

In [84]:
df.loc[df['time'] == '1950-01-01']

,time,latitude,longitude,temperature
2745600,1950-01-01,26.5,-122.5,NaN
2745601,1950-01-01,26.5,-121.5,NaN
2745602,1950-01-01,26.5,-120.5,NaN
2745603,1950-01-01,26.5,-119.5,NaN
2745604,1950-01-01,26.5,-118.5,NaN
...,...,...,...,...
2746739,1950-01-01,47.5,-75.5,2.540221
2746740,1950-01-01,47.5,-74.5,2.636371
2746741,1950-01-01,47.5,-73.5,2.642625
2746742,1950-01-01,47.5,-72.5,2.652048


In [85]:
df.isnull().sum()

time                 0
latitude             0
longitude            0
temperature    1148731
dtype: int64

In [86]:
3743168 - 1148731

2594437

In [87]:
df_copy = df.copy()

In [60]:
def split_time_series(start_year, end_year, data=df_copy, col='temperature'):
    data = data.loc[(data['time'] >= start_year) & ((data['time'] < end_year)), :]
    
    frame_to_use = data.loc[data[col].notna(), :]
    frame_to_pred = data.loc[data[col].isna(), :]
    
    X = frame_to_use.drop(['temperature', 'time'], axis=1)
    y = frame_to_use['temperature']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    prediction_val = model.predict(X_val)
    prediction_test = model.predict(X_test)
    
    frame_to_pred.drop(['temperature', 'time'], axis=1, inplace=True)
    frame_to_pred.loc[:, 'temperature'] = model.predict(frame_to_pred)
    
    return frame_to_pred

In [61]:
preds = split_time_series('1901-01-01', '1950-01-01')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15036\231636882.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.drop(['temperature', 'time'], axis=1, inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15036\231636882.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.loc[:, 'temperature'] = model.predict(frame_to_pred)


In [62]:
df_copy.loc[(df_copy['time'] >= '1901-01-01') & 
            (df_copy['time'] < '1950-01-01'), :] = df_copy.loc[(df_copy['time'] >= '1901-01-01') & 
            (df_copy['time'] < '1950-01-01'), :].fillna(preds, axis=0)

In [63]:
preds_1 = split_time_series('1950-01-01', '2001-01-01')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15036\231636882.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.drop(['temperature', 'time'], axis=1, inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15036\231636882.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.loc[:, 'temperature'] = model.predict(frame_to_pred)


In [64]:
df_copy.loc[(df_copy['time'] >= '1950-01-01') & 
            (df_copy['time'] < '2001-01-01'), :] = df_copy.loc[(df_copy['time'] >= '1950-01-01') & 
            (df_copy['time'] < '2001-01-01'), :].fillna(preds, axis=0)

In [65]:
preds_2 = split_time_series('2001-01-01', '2021-01-01')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15036\231636882.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.drop(['temperature', 'time'], axis=1, inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15036\231636882.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.loc[:, 'temperature'] = model.predict(frame_to_pred)


In [66]:
df_copy.loc[(df_copy['time'] >= '2001-01-01') & 
            (df_copy['time'] < '2021-01-01'), :] = df_copy.loc[(df_copy['time'] >= '2001-01-01') & 
            (df_copy['time'] < '2021-01-01'), :].fillna(preds, axis=0)

In [67]:
preds_3 = split_time_series('2021-01-01', '2022-08-01')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15036\231636882.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.drop(['temperature', 'time'], axis=1, inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15036\231636882.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_to_pred.loc[:, 'temperature'] = model.predict(frame_to_pred)


In [68]:
df_copy.loc[(df_copy['time'] >= '2021-01-01') & 
            (df_copy['time'] < '2022-08-01'), :] = df_copy.loc[(df_copy['time'] >= '2021-01-01') & 
            (df_copy['time'] < '2022-08-01'), :].fillna(preds, axis=0)

In [69]:
df_copy

,time,latitude,longitude,temperature
0,1750-01-01,26.5,-122.5,NaN
1,1750-01-01,26.5,-121.5,NaN
2,1750-01-01,26.5,-120.5,NaN
3,1750-01-01,26.5,-119.5,NaN
4,1750-01-01,26.5,-118.5,NaN
...,...,...,...,...
3743163,2022-08-01,47.5,-75.5,1.652320
3743164,2022-08-01,47.5,-74.5,1.732874
3743165,2022-08-01,47.5,-73.5,1.779766
3743166,2022-08-01,47.5,-72.5,1.833344


In [70]:
df_copy = df_copy.loc[df_copy['time'] >= '1901-01-01', :]

In [71]:
df_copy.isnull().sum()

time                0
latitude            0
longitude           0
temperature    177016
dtype: int64

In [72]:
df_copy.loc[df_copy['temperature'].isnull(), :]

,time,latitude,longitude,temperature
2745600,1950-01-01,26.5,-122.5,NaN
2745601,1950-01-01,26.5,-121.5,NaN
2745602,1950-01-01,26.5,-120.5,NaN
2745603,1950-01-01,26.5,-119.5,NaN
2745604,1950-01-01,26.5,-118.5,NaN
...,...,...,...,...
3743000,2022-08-01,44.5,-82.5,NaN
3743052,2022-08-01,45.5,-82.5,NaN
3743149,2022-08-01,47.5,-89.5,NaN
3743151,2022-08-01,47.5,-87.5,NaN


In [73]:
df_copy.loc[df_copy['time'] == '1950-01-01']

,time,latitude,longitude,temperature
2745600,1950-01-01,26.5,-122.5,NaN
2745601,1950-01-01,26.5,-121.5,NaN
2745602,1950-01-01,26.5,-120.5,NaN
2745603,1950-01-01,26.5,-119.5,NaN
2745604,1950-01-01,26.5,-118.5,NaN
...,...,...,...,...
2746739,1950-01-01,47.5,-75.5,2.540221
2746740,1950-01-01,47.5,-74.5,2.636371
2746741,1950-01-01,47.5,-73.5,2.642625
2746742,1950-01-01,47.5,-72.5,2.652048


In [74]:
3743168 - 177016 

3566152

In [88]:
def split_time_series(start_year, end_year, data, col='temperature', chunk_size=10000):
    data = data.loc[(data['time'] >= start_year) & (data['time'] < end_year), :]

    frame_to_use = data.loc[data[col].notna(), :]
    frame_to_pred = data.loc[data[col].isna(), :]

    def batch_generator(df):
        start_idx = 0
        end_idx = chunk_size
        while start_idx < len(df):
            yield df.iloc[start_idx:end_idx, :]
            start_idx = end_idx
            end_idx += chunk_size
            end_idx = min(end_idx, len(df))

    X = frame_to_use.drop(['temperature', 'time'], axis=1)
    y = frame_to_use['temperature']

    train_val_split = int(len(X) * 0.8)
    train_split = int(train_val_split * 0.75)

    X_train = batch_generator(X.iloc[:train_split, :])
    y_train = batch_generator(y.iloc[:train_split])
    X_val = batch_generator(X.iloc[train_split:train_val_split, :])
    y_val = batch_generator(y.iloc[train_split:train_val_split])
    X_test = X.iloc[train_val_split:, :]
    y_test = y.iloc[train_val_split:]

    model = RandomForestRegressor(n_estimators=100, random_state=0)

    for X_batch, y_batch in zip(X_train, y_train):
        model.fit(X_batch, y_batch)

    prediction_val = []
    for X_batch in X_val:
        prediction_val.extend(model.predict(X_batch))

    prediction_test = model.predict(X_test)

    frame_to_pred.drop(['temperature', 'time'], axis=1, inplace=True)
    frame_to_pred.loc[:, 'temperature'] = model.predict(frame_to_pred)

    return frame_to_pred

In [89]:
preds = split_time_series('1901-01-01', '1950-01-01', df_copy)
df_copy.loc[(df_copy['time'] >= '1901-01-16') & (df_copy['time'] < '1950-01-16'), :] = df_copy.loc[(df_copy['time'] >= '1901-01-16') & (df_copy['time'] < '1950-01-16'), :].fillna(preds, axis=0)

IndexingError: Too many indexers